<a href="https://colab.research.google.com/github/rkpe/03_Github/blob/master/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CROP Recommendation

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, export_graphviz 
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor

In [2]:
def loadData(filename):
  data = pd.read_csv(filename)
  return data

In [3]:
#Read in the data from the github repo, you should also have this saved locally...
crop = loadData('Crops.csv')


In [4]:
#Let's take a look...
crop.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 80 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Dist Code                                    2484 non-null   int64  
 1   Year                                         2484 non-null   int64  
 2   State Code                                   2484 non-null   int64  
 3   State Name                                   2484 non-null   object 
 4   Dist Name                                    2484 non-null   object 
 5   RICE AREA (1000 ha)                          2484 non-null   float64
 6   RICE PRODUCTION (1000 tons)                  2484 non-null   float64
 7   RICE YIELD (Kg per ha)                       2484 non-null   float64
 8   WHEAT AREA (1000 ha)                         2484 non-null   float64
 9   WHEAT PRODUCTION (1000 tons)                 2484 non-null   float64
 10  

In [5]:

cols =  ['Dist Code', 'Year', 'State Code', 'State Name' , 'Dist Name', 'RICE YIELD (Kg per ha)', 'WHEAT YIELD (Kg per ha)', 'SUGARCANE YIELD (Kg per ha)','MAIZE YIELD (Kg per ha)']
crop = crop[cols]

In [6]:
fert = loadData('Fertilizer_Consumption.csv')

In [7]:
fert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2481 entries, 0 to 2480
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Dist Code                            2481 non-null   int64  
 1   Year                                 2481 non-null   int64  
 2   State Code                           2481 non-null   int64  
 3   State Name                           2481 non-null   object 
 4   Dist Name                            2481 non-null   object 
 5   NITROGEN CONSUMPTION (tons)          2481 non-null   float64
 6   NITROGEN SHARE IN NPK (Percent)      2481 non-null   float64
 7   NITROGEN PER HA OF NCA (Kg per ha)   2481 non-null   float64
 8   NITROGEN PER HA OF GCA (Kg per ha)   2481 non-null   float64
 9   PHOSPHATE CONSUMPTION (tons)         2481 non-null   float64
 10  PHOSPHATE SHARE IN NPK (Percent)     2481 non-null   float64
 11  PHOSPHATE PER HA OF NCA (Kg pe

In [8]:
colsf = ['Dist Code', 'Year', 'NITROGEN SHARE IN NPK (Percent)', 'PHOSPHATE SHARE IN NPK (Percent)', 'POTASH SHARE IN NPK (Percent)']
fert = fert[colsf]

In [9]:
curr = pd.merge(crop, fert, how = 'left', on=['Dist Code','Year'])

In [10]:
np.where(curr.isnull() == True)

(array([752, 752, 752, 754, 754, 754, 755, 755, 755]),
 array([ 9, 10, 11,  9, 10, 11,  9, 10, 11]))

In [11]:
curr.iloc[752] #useless rows, let's drop

Dist Code                                    95
Year                                       2010
State Code                                    7
State Name                          Maharashtra
Dist Name                                Bombay
RICE YIELD (Kg per ha)                      0.0
WHEAT YIELD (Kg per ha)                     0.0
SUGARCANE YIELD (Kg per ha)                 0.0
MAIZE YIELD (Kg per ha)                     0.0
NITROGEN SHARE IN NPK (Percent)             NaN
PHOSPHATE SHARE IN NPK (Percent)            NaN
POTASH SHARE IN NPK (Percent)               NaN
Name: 752, dtype: object

In [12]:
curr  = curr.dropna()

In [13]:
curr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2481 entries, 0 to 2483
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Dist Code                         2481 non-null   int64  
 1   Year                              2481 non-null   int64  
 2   State Code                        2481 non-null   int64  
 3   State Name                        2481 non-null   object 
 4   Dist Name                         2481 non-null   object 
 5   RICE YIELD (Kg per ha)            2481 non-null   float64
 6   WHEAT YIELD (Kg per ha)           2481 non-null   float64
 7   SUGARCANE YIELD (Kg per ha)       2481 non-null   float64
 8   MAIZE YIELD (Kg per ha)           2481 non-null   float64
 9   NITROGEN SHARE IN NPK (Percent)   2481 non-null   float64
 10  PHOSPHATE SHARE IN NPK (Percent)  2481 non-null   float64
 11  POTASH SHARE IN NPK (Percent)     2481 non-null   float64
dtypes: flo

In [14]:
period = loadData('Crop_growing_period.csv')

In [15]:
period = period[['Dist Code', 'LENGTH OF GROWING PERIOD DAYS (Number)']]

In [16]:
period.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 2 columns):
 #   Column                                  Non-Null Count  Dtype
---  ------                                  --------------  -----
 0   Dist Code                               313 non-null    int64
 1   LENGTH OF GROWING PERIOD DAYS (Number)  313 non-null    int64
dtypes: int64(2)
memory usage: 5.0 KB


In [17]:
curr = pd.merge(curr, period, how = 'left', on=['Dist Code'])

In [18]:
curr = curr.replace(-1, np.NaN)

In [19]:
curr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2481 entries, 0 to 2480
Data columns (total 13 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Dist Code                               2481 non-null   int64  
 1   Year                                    2481 non-null   int64  
 2   State Code                              2481 non-null   int64  
 3   State Name                              2481 non-null   object 
 4   Dist Name                               2481 non-null   object 
 5   RICE YIELD (Kg per ha)                  2481 non-null   float64
 6   WHEAT YIELD (Kg per ha)                 2481 non-null   float64
 7   SUGARCANE YIELD (Kg per ha)             2481 non-null   float64
 8   MAIZE YIELD (Kg per ha)                 2481 non-null   float64
 9   NITROGEN SHARE IN NPK (Percent)         2481 non-null   float64
 10  PHOSPHATE SHARE IN NPK (Percent)        2481 non-null   floa

In [20]:
land = loadData('LandUse.csv')

In [21]:
land = land[['Dist Code','Year','TOTAL AREA (1000 ha)','NET CROPPED AREA (1000 ha)', 'GROSS CROPPED AREA (1000 ha)','CROPING INTENSITY (Percent)']]

In [22]:
land = land.replace(-1,0) #looked at data, it should be 0

In [23]:
land.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2381 entries, 0 to 2380
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Dist Code                     2381 non-null   int64  
 1   Year                          2381 non-null   int64  
 2   TOTAL AREA (1000 ha)          2381 non-null   float64
 3   NET CROPPED AREA (1000 ha)    2381 non-null   float64
 4   GROSS CROPPED AREA (1000 ha)  2381 non-null   float64
 5   CROPING INTENSITY (Percent)   2381 non-null   float64
dtypes: float64(4), int64(2)
memory usage: 111.7 KB


In [24]:
curr = pd.merge(curr, land, how = 'left', on=['Dist Code', 'Year'])

In [25]:
curr.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2481 entries, 0 to 2480
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Dist Code                               2481 non-null   int64  
 1   Year                                    2481 non-null   int64  
 2   State Code                              2481 non-null   int64  
 3   State Name                              2481 non-null   object 
 4   Dist Name                               2481 non-null   object 
 5   RICE YIELD (Kg per ha)                  2481 non-null   float64
 6   WHEAT YIELD (Kg per ha)                 2481 non-null   float64
 7   SUGARCANE YIELD (Kg per ha)             2481 non-null   float64
 8   MAIZE YIELD (Kg per ha)                 2481 non-null   float64
 9   NITROGEN SHARE IN NPK (Percent)         2481 non-null   float64
 10  PHOSPHATE SHARE IN NPK (Percent)        2481 non-null   floa

In [26]:
#just missing some years (2017), so let's average
list = ['TOTAL AREA (1000 ha)','NET CROPPED AREA (1000 ha)', 'GROSS CROPPED AREA (1000 ha)','CROPING INTENSITY (Percent)']
for x in list:
    curr[x] = curr.groupby("Dist Code")[x].transform(
        lambda x: x.fillna(x.mean()))

In [27]:
curr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2481 entries, 0 to 2480
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Dist Code                               2481 non-null   int64  
 1   Year                                    2481 non-null   int64  
 2   State Code                              2481 non-null   int64  
 3   State Name                              2481 non-null   object 
 4   Dist Name                               2481 non-null   object 
 5   RICE YIELD (Kg per ha)                  2481 non-null   float64
 6   WHEAT YIELD (Kg per ha)                 2481 non-null   float64
 7   SUGARCANE YIELD (Kg per ha)             2481 non-null   float64
 8   MAIZE YIELD (Kg per ha)                 2481 non-null   float64
 9   NITROGEN SHARE IN NPK (Percent)         2481 non-null   float64
 10  PHOSPHATE SHARE IN NPK (Percent)        2481 non-null   floa

In [28]:
rain = loadData('Monthly_Rainfall.csv')

In [29]:
rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1703 entries, 0 to 1702
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Dist Code                         1703 non-null   int64  
 1   Year                              1703 non-null   int64  
 2   State Code                        1703 non-null   int64  
 3   State Name                        1703 non-null   object 
 4   Dist Name                         1703 non-null   object 
 5   JANUARY RAINFALL (Millimeters)    1703 non-null   float64
 6   FEBRUARY RAINFALL (Millimeters)   1703 non-null   float64
 7   MARCH RAINFALL (Millimeters)      1703 non-null   float64
 8   APRIL RAINFALL (Millimeters)      1703 non-null   float64
 9   MAY RAINFALL (Millimeters)        1703 non-null   float64
 10  JUNE RAINFALL (Millimeters)       1703 non-null   float64
 11  JULY RAINFALL (Millimeters)       1703 non-null   float64
 12  AUGUST

In [30]:
rain = rain[['Dist Code', 'Year', 'ANNUAL RAINFALL (Millimeters)']]

In [31]:
curr1 = pd.merge(curr, rain, how = 'left', on=['Dist Code', 'Year'])

In [32]:
curr1.head(10)

,Dist Code,Year,State Code,State Name,Dist Name,RICE YIELD (Kg per ha),WHEAT YIELD (Kg per ha),SUGARCANE YIELD (Kg per ha),MAIZE YIELD (Kg per ha),NITROGEN SHARE IN NPK (Percent),PHOSPHATE SHARE IN NPK (Percent),POTASH SHARE IN NPK (Percent),LENGTH OF GROWING PERIOD DAYS (Number),TOTAL AREA (1000 ha),NET CROPPED AREA (1000 ha),GROSS CROPPED AREA (1000 ha),CROPING INTENSITY (Percent),ANNUAL RAINFALL (Millimeters)
0,1,2010,14,Chhattisgarh,Durg,1695.77,994.69,256.96,1457.32,54.47,31.79,13.75,164.0,1988.17,1028.600000,1405.630000,136.65,1149.7
1,1,2011,14,Chhattisgarh,Durg,1756.23,1115.68,506.02,1285.56,53.56,33.69,12.75,164.0,1988.17,1030.410000,1409.940000,136.83,1282.3
2,1,2012,14,Chhattisgarh,Durg,1900.97,1087.10,252.98,1538.56,58.43,31.47,10.11,164.0,1988.17,1029.070000,1415.980000,137.60,1092.1
3,1,2013,14,Chhattisgarh,Durg,1581.86,1043.06,237.45,1742.20,58.05,30.36,11.59,164.0,1639.34,1030.520000,1431.300000,138.89,1568.0
4,1,2014,14,Chhattisgarh,Durg,1729.89,1151.49,602.17,1451.66,56.70,33.33,9.97,164.0,1640.28,1026.150000,1438.760000,140.21,1379.7
5,1,2015,14,Chhattisgarh,Durg,1076.95,1120.74,228.49,1467.17,57.14,32.69,10.17,164.0,1640.28,1030.000000,1438.030000,139.61,1008.7
6,1,2016,14,Chhattisgarh,Durg,2085.96,1263.55,389.38,2028.48,54.12,35.04,10.85,164.0,1988.17,1038.210000,1467.940000,141.39,NaN
7,1,2017,14,Chhattisgarh,Durg,1168.92,1153.77,181.06,1956.03,56.47,33.03,10.49,164.0,1838.94,1030.422857,1429.654286,138.74,NaN
8,2,2010,14,Chhattisgarh,Bastar,1905.45,2000.00,117.65,2025.21,52.94,30.45,16.61,194.0,3905.76,836.850000,869.330000,103.88,1623.2
9,2,2011,14,Chhattisgarh,Bastar,1231.23,1872.18,629.63,2063.00,54.65,34.03,11.32,194.0,3905.76,813.450000,844.830000,103.86,1160.5


In [33]:
#missing 2016 and 2017 years so let's average
curr1['ANNUAL RAINFALL (Millimeters)'] = curr1.groupby("Dist Code")['ANNUAL RAINFALL (Millimeters)'].transform(
    lambda x: x.fillna(x.mean()))

In [34]:
curr1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2481 entries, 0 to 2480
Data columns (total 18 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Dist Code                               2481 non-null   int64  
 1   Year                                    2481 non-null   int64  
 2   State Code                              2481 non-null   int64  
 3   State Name                              2481 non-null   object 
 4   Dist Name                               2481 non-null   object 
 5   RICE YIELD (Kg per ha)                  2481 non-null   float64
 6   WHEAT YIELD (Kg per ha)                 2481 non-null   float64
 7   SUGARCANE YIELD (Kg per ha)             2481 non-null   float64
 8   MAIZE YIELD (Kg per ha)                 2481 non-null   float64
 9   NITROGEN SHARE IN NPK (Percent)         2481 non-null   float64
 10  PHOSPHATE SHARE IN NPK (Percent)        2481 non-null   floa

In [35]:
#lots of nulls in growing days but that has large affect on regressor so let's drop rows and keep that column
np.where(curr1.isnull() == True)

(array([ 752, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1888, 1889, 1890,
        1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898, 1899, 1900, 1901,
        1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912,
        1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923,
        1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934,
        1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945,
        1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956,
        1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967,
        1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978,
        1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
        1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
        2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
        2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022,
        2023, 2024, 2025, 2026, 2027, 

In [36]:
curr1 = curr1.dropna()

In [37]:
curr1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2240 entries, 0 to 2480
Data columns (total 18 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Dist Code                               2240 non-null   int64  
 1   Year                                    2240 non-null   int64  
 2   State Code                              2240 non-null   int64  
 3   State Name                              2240 non-null   object 
 4   Dist Name                               2240 non-null   object 
 5   RICE YIELD (Kg per ha)                  2240 non-null   float64
 6   WHEAT YIELD (Kg per ha)                 2240 non-null   float64
 7   SUGARCANE YIELD (Kg per ha)             2240 non-null   float64
 8   MAIZE YIELD (Kg per ha)                 2240 non-null   float64
 9   NITROGEN SHARE IN NPK (Percent)         2240 non-null   float64
 10  PHOSPHATE SHARE IN NPK (Percent)        2240 non-null   floa

In [38]:
curr1.to_csv("DS_Project1_data_merged.csv")

In [39]:
X= curr1.drop(columns=['RICE YIELD (Kg per ha)','WHEAT YIELD (Kg per ha)', 'SUGARCANE YIELD (Kg per ha)','MAIZE YIELD (Kg per ha)','Dist Code','Year','State Code','State Name','Dist Name'])
y= curr1['RICE YIELD (Kg per ha)']

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, random_state=21)
X_tune, X_test, y_tune, y_test = train_test_split(X_test,y_test,  train_size = 0.50, random_state=49)

In [ ]:
y_test.info()

In [47]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1568 entries, 323 to 1232
Data columns (total 9 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   NITROGEN SHARE IN NPK (Percent)         1568 non-null   float64
 1   PHOSPHATE SHARE IN NPK (Percent)        1568 non-null   float64
 2   POTASH SHARE IN NPK (Percent)           1568 non-null   float64
 3   LENGTH OF GROWING PERIOD DAYS (Number)  1568 non-null   float64
 4   TOTAL AREA (1000 ha)                    1568 non-null   float64
 5   NET CROPPED AREA (1000 ha)              1568 non-null   float64
 6   GROSS CROPPED AREA (1000 ha)            1568 non-null   float64
 7   CROPING INTENSITY (Percent)             1568 non-null   float64
 8   ANNUAL RAINFALL (Millimeters)           1568 non-null   float64
dtypes: float64(9)
memory usage: 122.5 KB


In [46]:
cl =DecisionTreeRegressor()
cl.fit(X_train, y_train)

DecisionTreeRegressor()

In [49]:
pred = cl.predict(X_test)

In [50]:
print(cl.score(X_test, y_test))

0.5023369895806851


In [51]:
print(cl.score(X_tune, y_tune))

0.49951863824223675


In [52]:
metrics.r2_score(y_test, pred)

0.5023369895806851

In [53]:
np.sqrt(metrics.mean_squared_error(y_test, pred))

799.1228075230812

In [54]:
print(metrics.mean_absolute_error(y_test, pred)) #nice!

530.1624404761906


In [55]:
varimp=pd.DataFrame(cl.feature_importances_,index = X.columns,columns=['importance']).sort_values('importance', ascending=False)
print(varimp)

                                        importance
PHOSPHATE SHARE IN NPK (Percent)          0.157773
TOTAL AREA (1000 ha)                      0.149521
CROPING INTENSITY (Percent)               0.135055
LENGTH OF GROWING PERIOD DAYS (Number)    0.133430
NET CROPPED AREA (1000 ha)                0.102063
GROSS CROPPED AREA (1000 ha)              0.100689
POTASH SHARE IN NPK (Percent)             0.089245
ANNUAL RAINFALL (Millimeters)             0.070920
NITROGEN SHARE IN NPK (Percent)           0.061305


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [57]:
clf = MultiOutputRegressor(RandomForestRegressor(max_depth=2, random_state=0))
clf.fit(X_train, y_train)

ValueError: ignored

In [ ]:
clf.predict(X_test[[0]])

In [ ]:
clf.score(X_test, y_test, sample_weight=None)